# PCAF Data Pipeline to ingest the data provided by EDGAR (fossil CO2 values)

The source excel file contains 5 sheets and parsed&transposed using the configuration data saved in the file "Edgar.ini".

The output first saved in a CSV file and afterwards copied in a dataframe and saved in a database table containing the fields "provider,attribute,value,unit".


In [3]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
# install pycountry to retrieve ISO codes
%pip install jedi==0.17.2
%pip install -U "ipython>=7.20"
%pip install pycountry
%pip install country-converter



You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 1.6 MB 35.7 MB/s            
     |████████████████████████████████| 136 kB 149.2 MB/s            
     |████████████████████████████████| 1.6 MB 141.8 MB/s            
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' comman

In [11]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

### Load Environment Variables

In [12]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [17]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_iceberg_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'pcaf_sovereign_edgar'

In [18]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_iceberg_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [19]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
Open a Trino connection using JWT for authentication

In [22]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [23]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epa_frs',)
('epa_ghgrp',)
('epacems',)
('epacems_y95_al',)
('esg_matching',)
('essd',)
('ghgrp_demo',)
('gleif',)
('gleif_mdt',)
('iceberg_demo',)
('information_schema',)
('ingest_schema',)
('iso3166',)
('itr_mdt',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('pcaf_sovereign_footprint',)
('physical_risk_project',)
('pudl',)
('rmi_20210929',)
('rmi_20211120',)
('rmi_20220119',)
('rmi_utility_transition_hub',)
('sec_dera',)
('sfi_geoasset',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('us_census',)
('wri',)
('wri_demo',)
('wri_dev',)
('wri_gppd',)
('wri_gppd_md',)
('wri_new',)
('wri_test',)


In [ ]:
Load GDP file (updated sporadically from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/EDGAR/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
ticker_file.download_file(f'/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
%run TransposeXLS.py --config Edgar.ini --output=EDGAR.csv 
df = pd.read_csv('EDGAR.csv')
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','unit']].dropna(subset=['value'])
#df["country_iso_code"]='###'
#df.validity_date = df.validity_date.astype(str)
df = df.convert_dtypes()
print(df.info(verbose=True))
df
#df


In [ ]:
# EDGAR does not deliver ISO codes , set the field using pycountry


import pycountry  as c
import country_converter as coco

def getCountryISOCode(country_name):
    try:
        country=c.countries.search_fuzzy(country_name)
        if country is None:
            print(country_name)
            return "###"
        else:
            return country[0].alpha_3
    except:
        print(country_name);
        return "###"

    
cc = coco.CountryConverter()

#some_names = ['Democratic Republic of the Congo']
#some_names = ['Germany']
df["country_iso_code"] = cc.convert(names = df["country_name"], to = 'ISO3')
df
    
#df["country_conv_name"] = df["country_name"].apply(lambda x: getCountryISOCode(x))

#c.countries.search_fuzzy('Democratic Republic of the Congo')   
#c.countries.search_fuzzy('Congo')   

#c.countries.search_fuzzy('Germany')   

    
#df["country_iso_code"] = df["country_name"].apply(lambda x: getCountryISOCode(x))
#df






In [29]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())



drop table if exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_edgar

[(True,)]


In [30]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_edgar(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date bigint,
    attribute varchar,
    value double,
    unit varchar
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)

[(True,)]


In [31]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(None,)]


In [35]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,unit
0,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AFG,Afghanistan,2010,fossil_CO2_totals_by_country,7.263269,Mt CO2/yr
1,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,ALB,Albania,2010,fossil_CO2_totals_by_country,4.599694,Mt CO2/yr
2,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,DZA,Algeria,2010,fossil_CO2_totals_by_country,116.529633,Mt CO2/yr
3,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AGO,Angola,2010,fossil_CO2_totals_by_country,23.836741,Mt CO2/yr
4,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AIA,Anguilla,2010,fossil_CO2_totals_by_country,0.024005,Mt CO2/yr
...,...,...,...,...,...,...,...,...
90148,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,EGY,Egypt,1973,fossil_CO2_totals_by_country,25.260349,Mt CO2/yr
90149,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,SLV,El Salvador,1973,fossil_CO2_totals_by_country,2.068914,Mt CO2/yr
90150,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,GNQ,Equatorial Guinea,1973,fossil_CO2_totals_by_country,0.032144,Mt CO2/yr
90151,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,ERI,Eritrea,1973,fossil_CO2_totals_by_country,0.341433,Mt CO2/yr


In [33]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
#df["country_iso_code"]='###'
#df.validity_date = df.validity_date.astype(str)
df.to_sql(ingest_table,
           con=engine, 
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 5000, verbose = True))

osc_datacommons_iceberg_dev
inserting 5000 records
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGAR', 'AFG', 'Afghanistan', 1970, 'fossil_CO2_totals_by_country', 1.7182055194, 'Mt CO2/yr')
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGAR', 'ALB', 'Albania', 1970, 'fossil_CO2_totals_by_country', 4.813550215686, 'Mt CO2/yr')
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGAR', 'DZA', 'Algeria', 1970, 'fossil_CO2_totals_by_country', 18.93157288567, 'Mt CO2/yr')
  ...
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGAR', 'EST', 'Estonia', 1973, 'fossil_CO2_totals_by_country', 26.8718873838, 'Mt CO2/yr')
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(5000,)]
inserting 5000 records
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGAR', 'SWZ', 'Eswatini', 1973, 'fossil_CO2_totals_by_country', 0.191316661, 'Mt CO2/yr')
  ('EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls', 'EDGA

In [34]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}" + "where validity_date=2020 and attribute='fossil_CO2_totals_by_country'"""
pd.read_sql(sql, engine)
#attribute='fossil_CO2_totals_by_country'


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value,unit
0,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AFG,Afghanistan,2020,fossil_CO2_totals_by_country,11.947864,Mt CO2/yr
1,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,ALB,Albania,2020,fossil_CO2_totals_by_country,5.103103,Mt CO2/yr
2,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,DZA,Algeria,2020,fossil_CO2_totals_by_country,163.473371,Mt CO2/yr
3,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AGO,Angola,2020,fossil_CO2_totals_by_country,22.513601,Mt CO2/yr
4,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,AIA,Anguilla,2020,fossil_CO2_totals_by_country,0.023859,Mt CO2/yr
...,...,...,...,...,...,...,...,...
205,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,YEM,Yemen,2020,fossil_CO2_totals_by_country,10.318484,Mt CO2/yr
206,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,ZMB,Zambia,2020,fossil_CO2_totals_by_country,7.500936,Mt CO2/yr
207,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,ZWE,Zimbabwe,2020,fossil_CO2_totals_by_country,11.558587,Mt CO2/yr
208,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,not found,International Aviation,2020,fossil_CO2_totals_by_country,337.116016,Mt CO2/yr
